In [1]:
import pandas as pd
import numpy as np
import googletrans
from googletrans import Translator
import time
import jieba
import random
import math

In [2]:
print(googletrans.LANGUAGES) #check language codes
print(googletrans.__version__) #make sure this is 4.0.0-rc.1
translator = Translator()
translator.raise_Exception = True

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [3]:
cs_data = pd.read_csv('redoing_cs_data.csv')
cs_data.head()

,sent_type,university,sent_id,aligned_to,original_sentence,eng_translation,zh_translation,cs_word_id,first_cs_word_form,first_cs_word_translation,...,surprisal_values,average_surprisal,bilingual_corpus_frequency_negative_log_first_cs_word,bilingual_corpus_frequency_negative_log_first_cs_word_trans,length_first_cs_word_form,dependency_distance,if_it_is_root,if_previous_word_is_punctuation,surprisal_of_previous_word,surprisal_first_cs_word_trans
0,code-switch,CMU,2,CMU_296,rich neighborhood ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 ...,Rich neighborhood is very safe (it belongs to ...,富人 区 ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 就 是 公园 ， 附近 还...,1,rich,富人,...,6.100998 1.320391 0.7630151 4.4722 3.20577 2.7...,3.144718,11.319098,10.402807,4,1,0,NaN,NaN,6.100998
1,code-switch,CMU,4,CMU_3006,lease 从 八月 开始 ， 具体 日期 可以 商量 。,Lease will begin in August and the date is neg...,租约 从 八月 开始 ， 具体 日期 可以 商量 。,1,lease,租约,...,6.527233 3.646975 4.193077 1.173965 0.5694761 ...,3.155464,7.348806,10.066335,5,8,0,NaN,NaN,6.527233
2,code-switch,CMU,5,CMU_70,house 一共 有 两 层 + 地下室 。,House has two floors plus basement.,房子 一共 有 两 层 + 地下室 。,1,house,房子,...,5.512905 5.228868 0.5955775 1.38895 2.136289 4...,3.268832,6.694125,7.084991,5,2,0,NaN,NaN,5.512905
3,code-switch,CMU,6,CMU_6777,一共 有 两 个 可以 洗澡 的 bathroom 。,There are two bathrooms for bathing.,一共 有 两 个 可以 洗澡 的 浴室 。,8,bathroom,浴室,...,4.509301 0.2903937 1.38895 0.4321387 3.465795 ...,2.642274,8.611048,9.709660,8,6,0,0.0,0.895995,5.210801
4,code-switch,CMU,9,CMU_4373,整个 house 家具 齐全 。,The whole house is fully furnished.,整个 房子 家具 齐全 。,2,house,房子,...,3.732348 4.488641 5.760885 6.166569 0.8237057,4.194430,6.694125,7.084991,5,1,0,0.0,3.732348,4.488641


In [4]:
sentences = list(cs_data['original_sentence'])
sentences

['rich neighborhood ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 就 是 公园 ， 附近 还有 配套 的 各 种 球场 和 社区 设施 ， 环境 很 好 很 安静 。',
 'lease 从 八月 开始 ， 具体 日期 可以 商量 。',
 'house 一共 有 两 层 + 地下室 。',
 '一共 有 两 个 可以 洗澡 的 bathroom 。',
 '整个 house 家具 齐全 。',
 '后院 ： deck 。',
 '空调 有 window ac 和 暖气 。',
 '入住 时间 flexible ， 可 长期 入住 。',
 '感兴趣 的 同学 ， 欢迎 email （ 请 注明 “ 租房 ” ） ， 请 简单 介绍 一下 自己 。',
 '二 楼 是 三 间 卧室 和 一 个 full 卫生间 。',
 '有 兴趣 者 ， 请 email 短信 介绍 一下 自己 情况 。',
 '半山 独 栋 别墅 ， 自 带 车库 和 driveway 。',
 '一 楼 有 half bath 。',
 '精装 修 家具 全 free 无线 上网 !',
 '对比 其他 地方 ， 可能 和 很多 人 share ， 并且 还 很 贵 ！',
 '房东 非常 无敌 nice 还 会 帮忙 丢 垃圾 ！',
 '户型 ： 一 楼 是 超大 living room ， dining room 及 厨房 ；',
 '长 租 all gone ！',
 '暑期 短 租 还 available 哦 。',
 '有 pet 可以 商量 正规 物业 管理 ， pa 法律 规定 租房 合同 ， 保护 双方 权益 ， 安心 入住 。',
 '欢迎 email 联系 咨询 。',
 '所有 想 寄 想 return 的 东西 ， 直接 放在 公寓 一 楼 就 可以 。',
 '免费 送 顶楼 view 和 顶楼 的 阳光 普照 !',
 'bathroom : 二 楼 和 地下室 各 有 一 个 卫生间',
 '适合 couple 或 单身 居住 。',
 '有 车库 ， off street parking .',
 '要求 ： no smoking ， no pets ， no parties .',
 'house 里面 有 一 个 shared

In [16]:
zh_translated_lines = []
for i in range(len(sentences)):
    if i%100==0:
        print(i)
    sent = sentences[i]
    try:
        result = translator.translate(sent, src='en', dest='zh-cn').text
        time.sleep(0.25)
        zh_translated_lines.append(result)
    except:
        zh_translated_lines.append('TODO')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


In [17]:
zh_translated_lines.count('TODO')

0

In [20]:
zh_translated_lines

['丰富的邻域，非常（属于富人），旁边就是，附近是是有的各球场社区社区社区社区，环境和',
 '租赁从八月，具体具体可以商量。',
 '房屋一共有层 +地下室地下室',
 '一共有有个可以洗澡的的。',
 '整整整家具家具。',
 '后院：甲板。',
 'window窗口ac和和暖气。',
 '入住时间柔性，可可长期入住。',
 '感兴趣感兴趣的，电子邮件（（请请“租”（“租”），请请简单介绍自己一下。',
 '二楼是间和和一一个一',
 '有有兴趣，电子邮件短信短信介绍自己自己。情况',
 '半山半山栋，车道。。。。',
 '一一楼半浴。',
 '精装精装修家具无线！',
 '对比其他，可能可能很多很多人很多人很多很多，并且并且并且',
 '房东房东无敌还还帮忙帮忙丢！',
 '户型：一一是超大超大大超大大超大超超',
 '长租都消失了！',
 '可用的哦哦哦',
 'PET可以可以正规物业，Pa法律法律房房，保护，保护双方，安心',
 '欢迎电子邮件联系联系咨询咨询',
 '返回的的的，直接直接放在一楼。可以。',
 '查看和和顶楼阳光阳光！',
 '浴室：二二和各有个个',
 '适合夫妇或或或居住。居住',
 '有，在路边停车场。',
 '要求：没有吸烟，没有宠物，没有聚会。',
 '房屋里面里面一个个共享厨房和餐厅，烤箱烤箱都都，餐具都都都都也',
 'to层层层帮忙开门和收发，不不担心快递丢失问题；',
 '八月开始可年年年',
 '性格性格好好，绝对绝对不打扰打扰',
 '［治安附近治安治安治安',
 '露天购物经历经历',
 '（对比其他，可能可能很多很多很多很多很多人很多很多很多很多很多',
 '附近邻居非常非常也非常，临近临近临近区区区',
 '房子房子一半家具，',
 '房子房子几乎几乎是',
 '1-2间间间带家具家具的家具家具家具家具或者或者或者',
 '房间房间一一的的，非常顶层的宽敞宽敞',
 '预算灵活灵活',
 '客厅，步入式壁橱也也大大大',
 '如果如果兴趣，请请电子邮件联系em@7',
 '租赁办公室就大厅，员工员工非常，有非常什么可以他们解决解决解决',
 '房东人房东人好。',
 '由由正规，欢迎电子邮件联系联系咨询详细详细',
 '卧室内家具，大号床，拎包，拎包拎包。。',
 '有室内和室外的的，而且的而且街区都免费街趴'

In [21]:
tokenized_zh_translated_lines = []
for sent in zh_translated_lines:
    tokenized_zh_translated_lines.append(' '.join((jieba.lcut(sent))))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/1p/_f5vztgn53n2ctrgr8fsk3b80000gn/T/jieba.cache
Loading model cost 0.517 seconds.
Prefix dict has been built successfully.


In [22]:
tokenized_zh_translated_lines

['丰富 的 邻域 ， 非常 （ 属于 富人 ） ， 旁边 就是 ， 附近 是 是 有 的 各 球场 社区 社区 社区 社区 ， 环境 和',
 '租赁 从 八月 ， 具体 具体 可以 商量 。',
 '房屋 一共 有层   + 地下室 地下室',
 '一共 有 有 个 可以 洗澡 的 的 。',
 '整整 整 家具 家具 。',
 '后院 ： 甲板 。',
 'window 窗口 ac 和 和 暖气 。',
 '入住 时间 柔性 ， 可可 长期 入住 。',
 '感兴趣 感兴趣 的 ， 电子邮件 （ （ 请 请 “ 租 ” （ “ 租 ” ） ， 请 请 简单 介绍 自己 一下 。',
 '二楼 是 间 和 和 一 一个 一',
 '有 有 兴趣 ， 电子邮件 短信 短信 介绍 自己 自己 。 情况',
 '半山 半山 栋 ， 车道 。 。 。 。',
 '一 一楼 半浴 。',
 '精装 精装修 家具 无线 ！',
 '对比 其他 ， 可能 可能 很多很多 人 很多 人 很多很多 ， 并且 并且 并且',
 '房东 房东 无敌 还 还 帮忙 帮忙 丢 ！',
 '户型 ： 一一 是 超大 超 大大超 大大超 大超超',
 '长租 都 消失 了 ！',
 '可用 的 哦哦哦',
 'PET 可以 可以 正规 物业 ， Pa 法律 法律 房房 ， 保护 ， 保护 双方 ， 安心',
 '欢迎 电子邮件 联系 联系 咨询 咨询',
 '返回 的 的 的 ， 直接 直接 放在 一楼 。 可以 。',
 '查看 和 和 顶楼 阳光 阳光 ！',
 '浴室 ： 二二 和 各有 个个',
 '适合 夫妇 或 或 或 居住 。 居住',
 '有 ， 在 路边 停车场 。',
 '要求 ： 没有 吸烟 ， 没有 宠物 ， 没有 聚会 。',
 '房屋 里面 里面 一个个 共享 厨房 和 餐厅 ， 烤箱 烤箱 都 都 ， 餐具 都 都 都 都 也',
 'to 层层 层 帮忙 开门 和 收发 ， 不 不 担心 快递 丢失 问题 ；',
 '八月 开始 可年 年 年',
 '性格 性格 好好 ， 绝对 绝对 不 打扰 打扰',
 '［ 治安 附近 治安 治安 治安',
 '露天 购物 经历 经历',
 '（ 对比 其他 ， 可能 可能 很多 很多很多 很多很多 人 很多很多 很多很多

In [23]:
file = open('calvillo_cs_data_automatic_google_translations_to_zh.txt','w')
for line in tokenized_zh_translated_lines:
    file.write(line+"\n")
file.close()

In [24]:
file = open('calvillo_cs_data_automatic_fanyi_translations_to_zh.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [34]:
lines[299] = ['大家可以查去年公布的犯罪警报。']
lines[599] = ['我希望找妹子火速去办公室商量这件事。']

In [35]:
lines

[['丰富的社区,非常安全(这里属于富人区),旁边就是公园,附近还有配套的各种球场和社区设施,环境很好很安静。'],
 ['请查阅从八月开始，具体日期可以商量。'],
 ['House一共有两层+地下室。'],
 ['一共有两个可以洗澡的浴室。'],
 ['整个house家具齐全。'],
 ['后院:甲板。'],
 ['空调有窗口ac和暖气。'],
 ['入住时间灵活，可长期入住。'],
 ['感兴趣的同学,欢迎邮件(请注明“租房”),请简单介绍一下自己。'],
 ['二楼是三间卧室和一个full卫生间。'],
 ['有兴趣者，请电子邮件短信介绍一下自己情况。'],
 ['半山独栋别墅，自带车库和车道。'],
 ['一楼有半浴。'],
 ['精装修家具全免费无线上网!'],
 ['对比其他地方，可能和很多人分享，并且还很贵!'],
 ['房东非常无敌不错还会帮忙丢垃圾!'],
 ['户型:一楼是超大客厅、餐厅及厨房;'],
 ['长租都没了!'],
 ['暑期短租还可用哦。'],
 ['有宠物可以商量正规物业管理,pa法律规定租房合同,保护双方权益,安心入住。'],
 ['欢迎电子邮件联系咨询。'],
 ['所有想寄想return的东西，直接放在公寓一楼就可以。'],
 ['免费送顶楼查看和顶楼的阳光普照!'],
 ['浴室:二楼和地下室各有一个卫生间'],
 ['适合夫妇或单身居住。'],
 ['有车库，远离街道停车。'],
 ['要求:禁止吸烟，禁止宠物，禁止派对。'],
 ['房子里面有一个共享厨房和餐厅,烤箱微波炉洗衣机都有,餐具也可以和家里其他人分享。'],
 ['一层有看门人帮忙开门和收发快递，不用担心快递丢失等问题;'],
 ['八月开始可签一年请。'],
 ['性格非常不错好相处，绝对不会打扰你。'],
 ['[治安]附近治安好，公寓有看门人，收发快递也方便。'],
 ['一个露天购物经历。'],
 ['(对比其他地方，可能和很多人分享，并且还很贵!'],
 ['附近邻居非常安静也非常安全，临近富人区。'],
 ['房子目前装修一半，'],
 ['房子目前几乎是装修,含电视,床,书桌,餐厅大桌子,椅子,生活用品基本齐全。'],
 ['想找1-2间带家具的房屋房或者公寓房，'],
 ['房间是一栋house的顶层，非常宽敞。'],
 ['预算灵活。']

In [36]:
tokenized_zh_translated_lines = []
for sent in lines:
    tokenized_zh_translated_lines.append(' '.join((jieba.lcut(sent[0]))))

In [37]:
len(tokenized_zh_translated_lines)

1476

In [38]:
file = open('calvillo_cs_data_automatic_fanyi_translations_to_zh.txt','w')
for line in tokenized_zh_translated_lines:
    file.write(line+"\n")
file.close()

In [7]:
file = open('calvillo_cs_data_automatic_google_translations_to_zh_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [8]:
lines = lines[:-4]

In [5]:
cs_word = list(cs_data['first_cs_word_translation'])

In [6]:
cs_word_id = list(cs_data['cs_word_id'])
cs_word_id = [num-1 for num in cs_word_id]

In [77]:
base_10_probs = ['TODO'] * len(lines)

for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    found = False
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == cs_word[i]:
            base_10_probs[i] = float(vals[j])
            found = True
            break
    if not found:
        try:
            base_10_probs[i] = float(vals[cs_word_id[i]])
            found = True
        except:
            pass
    if not found:
        for j in range(len(clean_words)):
            word = clean_words[j]
            if word.isascii():
                base_10_probs[i] = float(vals[j])
        
    if not found:
        base_10_probs[i] = float(vals[random.randint(0, len(words)-1)])

        

In [82]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps_google = [-val if type(val) is float else val for val in base_2_probs]

In [104]:
file = open('calvillo_cs_data_automatic_google_translations_to_zh_surps.txt','w')
for line in base_2_surps_google:
    file.write(str(line)+"\n")
file.close()

In [17]:
# for region after csw

base_10_probs = ['TODO'] * len(lines)

for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    found = False
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == cs_word[i]:
            try:
                base_10_probs[i] = float(vals[j+1])
                found = True
                break
            except:
                pass
    if not found:
        try:
            base_10_probs[i] = float(vals[cs_word_id[i+1]])
            found = True
        except:
            pass
    if not found:
        for j in range(len(clean_words)):
            word = clean_words[j]
            if word.isascii():
                try:
                    base_10_probs[i] = float(vals[j+1])
                except:
                    pass
        
#     if not found:
#         base_10_probs[i] = float(vals[-1])

In [18]:
base_10_probs.count('TODO')

171

In [19]:
base_10_probs_with_todos = base_10_probs
base_10_probs = [val for val in base_10_probs if type(val) is float]
len(base_10_probs)

1305

In [20]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps_google = [-val if type(val) is float else val for val in base_2_probs]

In [21]:
file = open('calvillo_cs_data_automatic_google_translations_to_zh_surps_after_csw.txt','w')
for line in base_2_surps_google:
    file.write(str(line)+"\n")
file.close()

In [22]:
file = open('calvillo_cs_data_automatic_fanyi_translations_to_zh_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [23]:
lines = lines[:-4]

In [13]:
base_10_probs = ['TODO'] * len(lines)

for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    found = False
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == cs_word[i]:
            base_10_probs[i] = float(vals[j])
            found = True
            break
    if not found:
        try:
            base_10_probs[i] = float(vals[cs_word_id[i]])
            found = True
        except:
            pass
    if not found:
        for j in range(len(clean_words)):
            word = clean_words[j]
            if word.isascii():
                base_10_probs[i] = float(vals[j])
        
    if not found:
        base_10_probs[i] = float(vals[random.randint(0, len(words)-1)])

In [89]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps_fanyi = [-val if type(val) is float else val for val in base_2_probs]

In [103]:
file = open('calvillo_cs_data_automatic_fanyi_translations_to_zh_surps.txt','w')
for line in base_2_surps_fanyi:
    file.write(str(line)+"\n")
file.close()

In [24]:
# for region after csw

base_10_probs = ['TODO'] * len(lines)

for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    found = False
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == cs_word[i]:
            try:
                base_10_probs[i] = float(vals[j+1])
                found = True
                break
            except:
                pass
    if not found:
        try:
            base_10_probs[i] = float(vals[cs_word_id[i+1]])
            found = True
        except:
            pass
    if not found:
        for j in range(len(clean_words)):
            word = clean_words[j]
            if word.isascii():
                try:
                    base_10_probs[i] = float(vals[j+1])
                except:
                    pass
        
#     if not found:
#         base_10_probs[i] = float(vals[-1])

In [25]:
base_10_probs.count('TODO')

118

In [26]:
base_10_probs_with_todos = base_10_probs
base_10_probs = [val for val in base_10_probs if type(val) is float]
len(base_10_probs)

1358

In [27]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps_fanyi = [-val if type(val) is float else val for val in base_2_probs]

In [28]:
file = open('calvillo_cs_data_automatic_fanyi_translations_to_zh_surps_after_csw.txt','w')
for line in base_2_surps_fanyi:
    file.write(str(line)+"\n")
file.close()

In [ ]:
# TODO: compare surprisal distributions to original surps --> redo R file